In [1]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow.keras.backend as K

from sificc_lib import AI, utils, Event, Simulation, root_files, DataModel

%matplotlib inline

In [2]:
# model name
model_name = 'model-2b-lsrz2-purity-FT1'

# source model name to load the network weights
source_model = 'model-2b-lsrz2-FT'

shuffle_clusters = False

# load the training data
data = DataModel('data-mtx-enough-top-6.npz', 
                 batch_size = 256, validation_percent = .05, test_percent = .1)

# append an extra dimention to the features since we are using convolutional layers
data.append_dim = True

# create an AI instance
ai = AI(data, model_name)

# randomly shuffle the training data
np.random.seed(888)
ai.data.shuffle(only_train=False)

# shuffle the clusters within each event
if shuffle_clusters:
    ai.data.shuffle_training_clusters()
    
# define the priority of selection
ai.data.weight_non_compton = .1

In [3]:
# define the learning rate scheduler
def lr_scheduler(epoch):
    if epoch < (7):
        return .00001
    else:
        return .000003

# define and create the neural network architecture
ai.create_model(conv_layers=[128, 64], classifier_layers=[32], type_layers=[8], 
                pos_layers=[64,32], energy_layers=[32, 16], base_l2=.0001, limbs_l2=.0001)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 54, 1)]      0                                            
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 6, 128)       1280        inputs[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv1D)                 (None, 6, 64)        8256        conv_1[0][0]                     
__________________________________________________________________________________________________
flatting (Flatten)              (None, 384)          0           conv_2[0][0]                     
______________________________________________________________________________________________

In [4]:
#LOADING
ai.load(model_name)
ai.compile_model()

In [ ]:
#TRAINING

In [4]:
# load the weights of the source model
ai.load(source_model, optimizer=False)
ai.compile_model()

# locate the correctly reconstructed events and replace the type target 
pred = ai.predict(ai.data.get_features())
true_type = ai.data._targets[:,0].copy()
sp_type = ai._find_matches(ai.data._targets, pred, keep_length=True)
ai.data._targets[:,0] = sp_type

In [6]:
%%time
# eliminate the components weight not intended for tuning
ai.weight_type = .05     * 1
ai.weight_e_cluster = .15* 0
ai.weight_p_cluster = .1 * 0
ai.weight_pos_x = 12     * 0
ai.weight_pos_y = 2      * 0
ai.weight_pos_z = 8      * 0
ai.weight_energy = 7     * 0

# freeze all network components
for layer in ai.model.layers:
    layer.trainable = False

# defreeze the parts to be tuned
for layer_name in ['dense_type_1', 'type']:
    layer = ai.model.get_layer(layer_name)
    layer.trainable = True

# print the trainable layers
for layer in ai.model.layers:
    if layer.trainable:
        print('{:17s}{}'.format(layer.name, layer.trainable))
print()
        
# compile the AI
ai.compile_model(learning_rate=.00001)
l_callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler),
]

# start the training of the network
ai.train(epochs=10, shuffle=True, shuffle_clusters=shuffle_clusters, 
         verbose=0, callbacks = l_callbacks)
print()

dense_type_1     True
type             True


CPU times: user 35min 6s, sys: 42.1 s, total: 35min 48s
Wall time: 35min 49s


In [9]:
# restore the true event type
ai.data._targets[:,0] = true_type

# evaluate the AI on the test set
ai.evaluate()

AI model
  Loss:        0.09476
    -Type:         0.41879 *  0.05 = 0.02094
    -Pos X:        0.00070 *  0.00 = 0.00000
    -Pos Y:        0.03808 *  0.00 = 0.00000
    -Pos Z:        0.00052 *  0.00 = 0.00000
    -Energy:       0.46219 *  0.00 = 0.00000
    -Cls e:        0.03397 *  0.00 = 0.00000
    -Cls p:        0.08168 *  0.00 = 0.00000
  Accuracy:    0.79508
    -TP rate:      0.41036
    -Cls e rate:   0.95384
    -Cls p rate:   0.88957
  Efficiency:  0.10191
  Purity:      0.19262
  Euc mean:    6.95769
  Euc std:    16.14937

Reco
  Accuracy:    0.64592
    -TP rate:      0.74924
  Efficiency:  0.10264
  Purity:      0.05959
  Euc mean:   20.44427
  Euc std:    48.46802


In [11]:
# save the trained model
ai.save(file_name=model_name)

In [5]:
# export the predictions for image reconstruction
ai.export_predictions_root(root_name='hyb_nn_cones_ptuned1.root')